# Hydroelastic Contact: Nonconvex Meshes
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).

# Check why Debug build is slow to load nonconvex VTK tetrahedral mesh

bazel run --compilation_mode=dbg //tutorials:hydroelastic_contact_nonconvex_debug_load

If you are not familiar with Drake's hydroelastic contact, study [hydroelastic_contact_basics.ipynb](./hydroelastic_contact_basics.ipynb). You can also find more information in Hydroelastic Contact User Guide [here.](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

## Introduction

This tutorial shows you how to set up dynamic simulations with hydroelastic contacts using nonconvex meshes. For context, we use a simple example of a compliant hydroelastic bell pepper dropped onto a compliant hydroelastic bowl on a compliant hydroelastic table top. Contact forces are calculated and visualized.

## Start MeshCat

See the section [Viewing models](./authoring_multibody_simulation.ipynb#Viewing-models) in the tutorial [Authoring a Multibody Simulation](./authoring_multibody_simulation.ipynb) for an introduction to MeshCat.

In [ ]:
%%time

from pydrake.geometry import StartMeshcat

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

## Create compliant-hydroelastic bell pepper in SDFormat

*Make sure you have the MeshCat tab opened in your browser;
the link is shown immediately above.*

We will create and visualize a compliant-hydroelastic bell pepper with SDFormat string.

We will use `ModelVisualizer` to verify the SDFormat description.
It will tell MeshCat to show the bell pepper.

### Visual geometry

First we create a visual geometry of the bell pepper. Observe in MeshCat the X(red), Y(green), and Z(blue) axes have origin near the bottom of the bell pepper. In general, a mesh geometry can have its origin anywhere.

Here MeshCat can show only the `drake/illustration` without `drake/proximity`.

In [ ]:
%%time

from pydrake.visualization import ModelVisualizer

visual_bell_pepper_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="BellPepper">
    <pose>0 0 0 0 0 0</pose>
    <link name="bell_pepper">
      <visual name="yellow_bell_pepper_no_stem">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(visual_bell_pepper_sdf, "sdf")
visualizer.Run(loop_once=True)

##### Collision geometry with hydroelastic properties

We will add the `<collision>` block with hydroelastic properties. Without `<collision>`, our object cannot make contact in the simulation.

Drake also uses the term proximity for collision. This is the `<drake:proximity_properties>` block that control hydroelastic contacts for SDFormat:

        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e6</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
        </drake:proximity_properties>

Use `<drake:compliant_hydroelastic/>` to specify a compliant-hydroelastic object.

We set `<drake:hydroelastic_modulus>` to 1e6 Pascals. It is comparable to high-density polyethylene (HDPE) with 1 GPa Young's modulus. Hydroelastic modulus is not a physical property but rather a numerical parameter to tune our contact model. As a rule of thumb, set hydroelastic modulus to about 1/100 of Young's modulus. 

We set `<drake:mu_dynamic>` (unitless) to 0.5 for the coefficient of dynamic (i.e., kinetic) friction, and set `<drake:hunt_crossley_dissipation>` to 1.25 seconds/meter as the dissipation constant for the Hunt & Crossley model.

Here MeshCat can show both `drake/illustration` and `drake/proximity` but `drake/inertia` is wrong. Toggle `drake/illustration` and `drake/proximity`on and off.

In [ ]:
%%time 

from pydrake.visualization import ModelVisualizer

collision_bell_pepper_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="BellPepper">
    <pose>0 0 0 0 0 0</pose>
    <link name="bell_pepper">
      <collision name="collision">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.vtk</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e6</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25 </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
      <visual name="visual">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/yellow_bell_pepper_no_stem_low.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(collision_bell_pepper_sdf, "sdf")
visualizer.Run(loop_once=True)

| load OBJ  | load OBJ + VTK |
| --------- | -------------- |
|  980ms    |  13.6s         |   